In [ ]:
!cd data
!git init
!git add .
!git remote add origin https://github.com/Rt39/BigDataProjAmazReview.git
!git commit -am "added"
!git branch -M main
!git push origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
import pyspark.sql.functions as F
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

In [ ]:
if not ("spark" in locals() or "spark" in globals()):
    conf = pyspark.SparkConf().setAppName('hw3').setMaster('local[*]')
    sc = pyspark.SparkContext(conf=conf)
    spark = pyspark.sql.SparkSession(sc)

In [ ]:
!wget "https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics_5.json.gz"
!wget "https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics.csv"
!gzip -d Electronics_5.json.gz

--2023-12-13 21:44:45--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics_5.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1251876861 (1.2G) [application/x-gzip]
Saving to: ‘Electronics_5.json.gz’

Electronics_5.json. 100%[===================>]   1.17G  61.7MB/s    in 19s     

2023-12-13 21:45:03 (64.3 MB/s) - ‘Electronics_5.json.gz’ saved [1251876861/1251876861]

--2023-12-13 21:45:03--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Electronics.csv
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 876247561 (836M) [text/csv]
Saving to: ‘Electronics.csv’

Electronics.csv

In [ ]:
# fix error in json reading
with open("Electronics_5.json", "r") as infile, open("Electronics_5_fixed.json", "w") as outfile:
    for line in infile:
        try:
            json_line = json.loads(line)
            if "style" in json_line:
                json_line.pop("style")
            if "Style" in json_line:
                json_line.pop("Style")
            json.dump(json_line, outfile)
            outfile.write("\n")
        except json.decoder.JSONDecodeError:
            print("Error in line: ", line)

electronics_core_df = spark.read.json("Electronics_5_fixed.json")

In [ ]:


electronics_ratings_df = spark.read.csv("Electronics.csv", header=True, inferSchema=True)

# New Section

In [ ]:
electronics_core_df.show(5)

+----------+-----+-------+--------------------+-----------+--------------+----------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|    reviewerName|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+----------------+--------------------+--------------+--------+----+
|0151004714| NULL|    5.0|This is the best ...|09 18, 1999| AAP7PPBU72QFM|    D. C. Carrad|      A star is born|     937612800|    true|  67|
|0151004714| NULL|    3.0|Pages and pages o...|10 23, 2013|A2E168DTVGE6SV|             Evy|A stream of consc...|    1382486400|    true|   5|
|0151004714| NULL|    5.0|This is the kind ...| 09 2, 2008|A1ER5AYS3FQ9O3|           Kcorn|I'm a huge fan of...|    1220313600|   false|   4|
|0151004714| NULL|    5.0|What gorgeous lan...| 09 4, 2000|A1T17LMQABMBN5| Caf Girl Writes|The most beautifu...|     968025600|   false|  13|
|01510

In [ ]:
electronics_ratings_df.show(5)

+----------+--------------+---+----------+
|0060009810|A1N070NS9CJQ2I|5.0|1026864000|
+----------+--------------+---+----------+
|0060009810|A3P0KRKOBQK1KN|5.0|1025913600|
|0060009810|A192HO2ICJ75VU|5.0|1025654400|
|0060009810|A2T278FKFL3BLT|4.0|1025395200|
|0060009810|A2ZUXVTW8RXBXW|5.0|1025222400|
|0060009810|A21JDG4HA6OLPF|4.0|1024963200|
+----------+--------------+---+----------+
only showing top 5 rows



In [ ]:
!head Electronics.csv

0060009810,A1N070NS9CJQ2I,5.0,1026864000
0060009810,A3P0KRKOBQK1KN,5.0,1025913600
0060009810,A192HO2ICJ75VU,5.0,1025654400
0060009810,A2T278FKFL3BLT,4.0,1025395200
0060009810,A2ZUXVTW8RXBXW,5.0,1025222400
0060009810,A21JDG4HA6OLPF,4.0,1024963200
0060009810,A2NCADLQY962YD,5.0,1024704000
0060786817,A2UNA438B0FRA8,5.0,1138233600
0060786817,A2NQQSHSQKKGEX,5.0,1137801600
0060786817,A5EXND10WD4PM,3.0,1137542400
